# Feature Engineering

### Libraries

In [1]:
# import libraries

import pandas as pd

### Data

In [2]:
# load negative, neutral and positive time series

negative_review_ts = pd.read_csv('data/negative_review_ts.csv', index_col=0, parse_dates=True)
neutral_review_ts = pd.read_csv('data/neutral_review_ts.csv', index_col=0, parse_dates=True)
positive_review_ts = pd.read_csv('data/positive_review_ts.csv', index_col=0, parse_dates=True)

In [3]:
# load sentiment time series

sentiment_ts = pd.read_csv('data/sentiment_ts.csv', index_col=0, parse_dates=True)

sentiment_ts

,review_id,sentiment
date,,
2011-01-08 23:52:22,NvusujU9_5pIUbn9SZ6hMA,1
2018-06-09 01:37:02,vHOeBa7aMA_na4rfS2Db5A,-1
2010-03-13 22:33:10,hG9RTxxivb0ZXzEk4JXTXA,-1
2017-12-16 03:11:50,zIVkwgahZjOneChZFUYY4g,1
2015-04-28 20:29:40,DLczAuvMAlAnY5EeDGhTVg,1
...,...,...
2014-08-16 02:47:47,OgoBp7fbXnLSKvsQb4O_tw,1
2021-12-20 11:58:50,Q7e8EtZMmdknDrQE7huMoQ,1
2019-10-04 21:57:19,zzMW6zbsFaQMjoGu2bGVdA,1


### Adjustments

In [ ]:
# TODO: check if adjustments are needed (e.g. adjusting inflation in monetary time series, mathematical transformations (box-cox), etc.)

### Decomposition

In [ ]:
# TODO: try decomposing the time series to check for seasonality and trends